In [5]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
import os
import cv2
import numpy as np

In [8]:
image_folder = "/content/drive/MyDrive/UP/VisaoComputacional/AtividadesAvaliativas/AtvAvaliativa1_2Bim/img_fruta"
folders = ['maça', 'limao', 'macadamia', 'blueberry', 'baunilha']

descriptors = []

for folder in folders:
    folder_path = os.path.join(image_folder, folder)
    image_files = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]

    if not image_files:
        print("The folder {} is empty. Continuing to the next folder.".format(folder))
        continue

    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        try:
            image = cv2.imread(image_path)
            if image is None:
                raise Exception("Error reading the image {}. Continuing to the next image.".format(image_path))

            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(gray, (64, 128))

            block_size = (16, 16)
            block_stride = (8, 8)
            cell_size = (8, 8)
            num_bins = 9

            hog = cv2.HOGDescriptor((64, 128), block_size, block_stride, cell_size, num_bins)

            descriptor = hog.compute(resized)

            descriptors.append(descriptor)
        except Exception as e:
            print(str(e))


In [9]:
import docx
from docx.shared import Pt

output_file = "/content/drive/MyDrive/UP/VisaoComputacional/AtividadesAvaliativas/AtvAvaliativa1_2Bim/img_fruta/descritores_frutas.docx"

processed_folders = set()

doc = docx.Document()

for i in range(len(folders)):
    folder = folders[i]
    descriptor = descriptors[i]
    label = folder.split('/')[-1]

    if label in processed_folders:
        continue

    processed_folders.add(label)

    doc.add_paragraph("Descritor da primeira imagem da pasta {}: ".format(label))
    doc.add_paragraph(','.join(map(str, descriptor)))

doc.save(output_file)

In [10]:
import docx
from docx.shared import Pt
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

labels = []
for i, folder in enumerate(folders):
    num_images = len(os.listdir(os.path.join(image_folder, folder)))
    labels.extend([folder] * num_images)

X_train, X_test, y_train, y_test = train_test_split(descriptors, labels, test_size=0.2, random_state=42)

svm = LinearSVC()
svm.fit(X_train, y_train)

y_predicted = svm.predict(X_test)

doc = docx.Document()

for true_label, pred_label in zip(y_test, y_predicted):
    doc.add_paragraph("Fruta Correta: {} ----> Fruta predita: {}".format(true_label, pred_label))

output_file = "/content/drive/MyDrive/UP/VisaoComputacional/AtividadesAvaliativas/AtvAvaliativa1_2Bim/img_fruta/results.docx"
doc.save(output_file)

In [11]:
import docx
from docx.shared import Pt
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix

accuracy = accuracy_score(y_test, y_predicted)
precision = precision_score(y_test, y_predicted, average='weighted')
confusion_mat = confusion_matrix(y_test, y_predicted)

doc = docx.Document()

doc.add_paragraph("Acuracia (%): {:.2f}%".format(accuracy * 100))
doc.add_paragraph("Precisão (%): {:.2f}%".format(precision * 100))
doc.add_paragraph("Matriz de confusão:")
for row in confusion_mat:
    doc.add_paragraph(" ".join(str(val) for val in row))

output_file = "/content/drive/MyDrive/UP/VisaoComputacional/AtividadesAvaliativas/AtvAvaliativa1_2Bim/img_fruta/metricas.docx"
doc.save(output_file)